In [2]:
from builtins import range, map, zip, filter
from io import open
import six,imp,os,sys
from six.moves import cPickle
import numpy as np 
# import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist,cifar10,fashion_mnist
from keras.models import Sequential,Model,load_model
from keras.layers import Dense, Dropout, Activation, Flatten,Conv1D, Conv2D, Convolution2D, MaxPooling1D,MaxPooling2D,GlobalAveragePooling2D
from keras.layers import CuDNNGRU,CuDNNLSTM,LSTM
from keras.layers.normalization import BatchNormalization
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet121, DenseNet169, DenseNet201,VGG16,ResNet50,InceptionV3,imagenet_utils
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.utils import plot_model
from keras.losses import MAE
from keras.optimizers import SGD, Adam, adam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow import set_random_seed
from PIL import Image
from numpy.random import seed
# seed(639)
# set_random_seed(5944)

Using TensorFlow backend.


# h5转Bson格式保存

In [6]:

# 保证所有数据能够显示，而不是用省略号表示，np.inf表示一个足够大的数
# np.set_printoptions(threshold = np.inf) 
# 若想不以科学计数显示:
# np.set_printoptions(suppress = True)
# np.set_printoptions(linewidth = np.inf)

import bson,re
class Convert2MyTools:
    dt = dict()
    layers = []
    def extract_con2d(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['filters'] = layer.filters
        obj['kernel_size'] = list(layer.kernel_size)
        obj['strides'] = list(layer.strides)
        #   tf中padding=='same'表示全填充,valid表示不填充，padding=0表示不填充,padding=1四周都填充
        obj['padding'] = 1 if layer.padding.lower()=='same' else 0
        obj['use_bias'] = layer.use_bias
        obj['activation'] = layer.activation.__name__
        weights = layer.get_weights()
        ws = weights[0]
        # tf中的纬度是h,w,inchannel,outchannel，在内部会根据批次转换成NHWC
        # Julia中计算是H W IC OC，julia中reshape方向相反,,python纬度转换变成(outchannel,inchannel,w,h) 拉平导出
        (filter_height, filter_width, in_channels, out_channels) = ws.shape
        obj['filters_shape'] = list(map(int,ws.shape)) # 这里的过滤器shape是为了后续的weights转换所用
        obj['filter_height'] = filter_height
        obj['filter_width'] = filter_width
        obj['in_channels'] = in_channels
        obj['out_channels'] = out_channels
        print(obj)
        obj['weights'] = ws.transpose(3,2,1,0).flatten().tolist()
        obj['bias'] = []
        if layer.use_bias:
            obj['bias'] = weights[1].tolist()
        data.append(obj)

    def extract_max_avg_pooling(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['pool_size'] = list(layer.pool_size)
        obj['strides'] = list(layer.strides)
        obj['padding'] = 1 if layer.padding.lower()=='same' else 0
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['activation'] = 'linear'
        print(obj)
        data.append(obj)
        
    def extract_global_max_avg_pooling(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['activation'] = 'linear'
        print(obj)
        data.append(obj)

    def extract_flatten(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['activation'] = 'linear'
        print(obj)
        data.append(obj)
    
    def extract_permute(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['dims'] = list(layer.dims)
        obj['activation'] = 'linear'
        print(obj)
        data.append(obj)

    def extract_dense(self,data,layer):
        weights = layer.get_weights()
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['units'] = layer.units
        obj['use_bias'] = layer.use_bias
#         if "activation" in layer.__dict__.items(): layer.has_key("activation")
        obj['activation'] = layer.activation.__name__
        # 由于weights[0][:,i]是一列作为特征进行相乘，所以横向拉平后的数据在julia中对应的shape需要转置
        obj['weights_shape'] = list(map(int,weights[0].shape))
        obj['weights_shape'].reverse()
        print(obj)
        obj['weights'] = weights[0].flatten().tolist()
        obj['bias'] = []
        if layer.use_bias:
            obj['bias'] = weights[1].tolist()
        data.append(obj)
     
    def extract_zeropadding(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['padding'] = np.array(layer.padding).flatten().tolist()
        obj['activation'] = 'linear'
        print(obj)
        data.append(obj)
    
    def extract_batchnorm(self,data,layer):
        weights = layer.get_weights()
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['momentum'] = float(layer.momentum)
        obj['epsilon'] = float(layer.epsilon)
        obj['center'] = layer.center
        obj['scale'] = layer.scale
        print(obj)
        obj['gamma'] = weights[0].tolist()
        obj['beta'] = weights[1].tolist()
        obj['moving_mean'] = weights[2].tolist()
        obj['moving_variance'] = weights[3].tolist()
        obj['activation'] = 'linear'
        data.append(obj)
        
    def extract_activation(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['activation'] = layer.activation.__name__
        print(obj)
        data.append(obj)
        
    def extract_add_or_concatenate(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['output_name'] = layer.output.name
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['activation'] = 'linear'
        obj['input_names'] = []
        obj['input_shapes'] = []
        for _input in layer.input:
            obj['input_names'].append(_input.name)
            obj['input_shapes'].append(_input.shape.as_list()[1:4])
        print(obj)
        data.append(obj)
        
    def extract_dropout(self,data,layer):
        obj = dict()
        obj['layer_name'] = layer.name
        obj['op_type'] = type(layer).__name__
        obj['input_name'] = layer.input.name
        obj['output_name'] = layer.output.name
        obj['input_shape'] = list(map(int,layer.input_shape[1:4]))
        obj['output_shape'] = list(map(int,layer.output_shape[1:4]))
        obj['rate'] = float(layer.rate)
        obj['activation'] = 'linear'
        print(obj)
        data.append(obj)
        
    global gd
    def extract_model(self,model,path):
        with open(path, 'wb') as fs:
            data=[]
            for index,layer in enumerate(model.layers):
                layer_name = type(layer).__name__
                if 'Conv2D' in layer_name:
                    self.extract_con2d(data,layer)
                elif 'Dense' in layer_name:
                    self.extract_dense(data,layer)
                elif 'Permute' in layer_name:
                    self.extract_permute(data,layer)
                elif 'Flatten' in layer_name:
                    self.extract_flatten(data,layer)
                elif 'Dropout' in layer_name:
                    self.extract_dropout(data,layer)
                elif 'Activation' in layer_name:
                    self.extract_activation(data,layer)
                elif 'ZeroPadding2D' in layer_name:
                    self.extract_zeropadding(data,layer)
                elif 'BatchNormalization' in layer_name:
                    self.extract_batchnorm(data,layer) 
                elif layer_name in ['Add','Concatenate']:
                    self.extract_add_or_concatenate(data,layer)
                elif layer_name in ['MaxPooling2D','AveragePooling2D']:
                    self.extract_max_avg_pooling(data,layer)
                elif layer_name in ['GlobalMaxPooling2D','GlobalAveragePooling2D']:
                    self.extract_global_max_avg_pooling(data,layer)
                elif 'InputLayer' in layer_name:
                    continue
                else:
                    print('not support layer',layer.name)
            fs.write(bson.dumps({'data':data}))
            print('export successfully!')


model = load_model('../../Neurify/mnist/models/各模型转换/neurify_convnet_kolter.h5')

a = Convert2MyTools()
a.extract_model(model,'neurify_convnet_kolter.bson')


# layer = model.layers[8]
# print(type(layer).__name__)
# print(layer.input_shape)
# print(layer.output_shape)
# weights = layer.get_weights()
# ws = weights[0].transpose(3,2,1,0).flatten()
# print(ws[:,:,0,0])
# w_t = ws.transpose(3,2,1,0).flatten().tolist()
# print(w_t)
# a = re.sub(r'(]\n)', '];',w_t)
# a = re.sub(r'\];[\s]*\[', '];[',a)
# for layer in model.layers:
#     print(layer.name)
# for item in layer.__dict__.items():
#     print(item)
# with open('tt.bson', 'wb') as fs:
#     p = dict()
#     p["data"]= ws.tolist()
#     fs.write(bson.dumps(p))
    

{'layer_name': 'conv2d_7', 'op_type': 'Conv2D', 'input_name': 'conv2d_7_input:0', 'output_name': 'conv2d_7/Relu:0', 'input_shape': [28, 28, 1], 'output_shape': [14, 14, 16], 'filters': 16, 'kernel_size': [4, 4], 'strides': [2, 2], 'padding': 1, 'use_bias': True, 'activation': 'relu', 'filters_shape': [4, 4, 1, 16], 'filter_height': 4, 'filter_width': 4, 'in_channels': 1, 'out_channels': 16}
{'layer_name': 'conv2d_8', 'op_type': 'Conv2D', 'input_name': 'conv2d_7/Relu:0', 'output_name': 'conv2d_8/Relu:0', 'input_shape': [14, 14, 16], 'output_shape': [7, 7, 32], 'filters': 32, 'kernel_size': [4, 4], 'strides': [2, 2], 'padding': 1, 'use_bias': True, 'activation': 'relu', 'filters_shape': [4, 4, 16, 32], 'filter_height': 4, 'filter_width': 4, 'in_channels': 16, 'out_channels': 32}
{'layer_name': 'permute_4', 'op_type': 'Permute', 'input_name': 'conv2d_8/Relu:0', 'output_name': 'permute_4/transpose:0', 'input_shape': [7, 7, 32], 'output_shape': [32, 7, 7], 'dims': [3, 1, 2], 'activation': '

In [4]:
def preprocessing_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    print(x_train.shape)
     # 归一化
    x_train = x_train / 255
    x_test = x_test / 255
    x_train = x_train.reshape(60000,784) # 将图片摊平，变成向量
    x_test = x_test.reshape(10000,784) # 对测试集进行同样的处理
    # 标签激活
    y_train = keras.utils.to_categorical(y_train, 10)
    y_test = keras.utils.to_categorical(y_test, 10)
    return x_train, y_train, x_test, y_test

In [5]:
model = Sequential()
model.add(Dense(units=10,activation='relu',input_shape=(784,)))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=10,activation='relu'))
model.add(Dense(units=10,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                7850      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_5 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_6 (Dense)              (None, 10)                110       
Total params: 8,400
Trainable params: 8,400
Non-trainable params: 0
____________________________________________________

In [7]:
x_train, y_train, x_test, y_test = preprocessing_data()
model.fit(x_train, y_train,batch_size=128, epochs=20, verbose=1, validation_data=(x_test, y_test))

(60000, 28, 28)

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 43us/step - loss: 1.2591 - accuracy: 0.5600 - val_loss: 0.5927 - val_accuracy: 0.8181
Epoch 2/20
60000/60000 [==============================] - 1s 19us/step - loss: 0.5137 - accuracy: 0.8454 - val_loss: 0.4514 - val_accuracy: 0.8665
Epoch 3/20
60000/60000 [==============================] - 1s 20us/step - loss: 0.4208 - accuracy: 0.8763 - val_loss: 0.3756 - val_accuracy: 0.8909
Epoch 4/20
60000/60000 [==============================] - 1s 20us/step - loss: 0.3679 - accuracy: 0.8928 - val_loss: 0.3443 - val_accuracy: 0.9040
Epoch 5/20
60000/60000 [==============================] - 1s 20us/step - loss: 0.3271 - accuracy: 0.9058 - val_loss: 0.3162 - val_accuracy: 0.9057
Epoch 6/20
60000/60000 [==============================] - 1s 20us/step - loss: 0.2973 - accuracy: 0.9134 - val_loss: 0.2906 - val_accuracy: 0.9166
Epoch 7/20
60000/60000 [===========================